### 참고

https://www.kaggle.com/code/robikscube/tutorial-time-series-forecasting-with-xgboost/notebook
    
https://github.com/ykpgrr/Price-Prediction-with-Random-Forest/blob/master/Bitcoin%20Price%20Prediction%20with%20Scikit-Learn%20using%20Random%20Forest.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
df

In [ ]:
# unixtimestamp 변환
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

# 결측치 채우기 : method = 'ffill' : 결측값을 바로 위 값과 동일하게 변경

df = df.fillna(method='ffill')
df.head()

In [ ]:
# 2017년 3월부터 2021년 3월까지 데이터 필터링

filtered_data = df[df['Timestamp'].between('2017-03-01','2021-03-31')]
filtered_data = df.set_index('Timestamp')
filtered_data

In [ ]:
filtered_data.info()

In [ ]:
# 연도별 price 시각화
_ = filtered_data['Weighted_Price'].plot(style='', figsize=(10,5),color='red', label='Weighted_Price')
plt.xlabel('Timestamp')
plt.title('연도별 Weighted_Price')
plt.legend()
plt.show()

### 데이터셋 구분

2020년도 이전을 학습 데이터로, 2020년도 이후를 예측 데이터로 사용하고자 함

In [ ]:
# 데이터셋 구분

split_date = pd.to_datetime('2020-03-01')
train = filtered_data.loc[filtered_data.index < split_date].copy()
test = filtered_data.loc[filtered_data.index >= split_date].copy()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 피처, 타겟으로 데이터 분리

features = train.drop('Weighted_Price', axis=1)
target = train['Weighted_Price']

X_train, X_test, y_train, y_test = train_test_split(features,
                                                   target,
                                                   test_size=0.2,
                                                   random_state=42)

# 랜덤 포레스트 모델 학습시키기
rf = RandomForestRegressor(n_estimators=1000, random_state=5)
rf.fit(X_train, np.ravel(y_train))
predictions = rf.predict(X_test)

In [ ]:
# 결과 데이터프레임

Result = pd.DataFrame(y_test, index= X_test.index, columns=['NEXT_CLOSE'])
Result['Predicted'] = predictions
Result = Redult.sort_values('Timestamp')
Result.head()

In [ ]:
# 예측 결과 시각화

plt.figure(figsize=(10,5))
ax = plt.gca()

plt.plot(Result['NEXT_CLOSE'], color= 'r', label='Real BTC Price')
plt.plot(Result['Predicted'], color='b', label= 'Predicted BTC Price')
plt.title('BTC Price prediction')
plt.xlabel('Time')
plt.legend()
plt.show()